In [12]:
import pandas as pd
from pathlib2 import Path
from butcherbird.utils.paths import DATA_DIR
SAVE_LOC = DATA_DIR/'interim/raw_spectrogram/butcherbird_note_df.pickle'
note_df = pd.read_pickle(SAVE_LOC)
note_df.head()

,phrase_nb,phrase_strt,phrase_end,phrase_len,note_cnt,note_nb,note_strt,note_end,note_len,indv,indvi,key,audio,rate,spectrogram
0,0,513.78,517.061923,3.281923,15,0,513.780000,513.905214,0.125214,Bird5,0,LS110143.10.12.2015GeoGreens_dn,"[-0.0, -7.356902671940375e-05, -0.000162745769...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,0,513.78,517.061923,3.281923,15,1,513.917747,513.980936,0.063188,Bird5,0,LS110143.10.12.2015GeoGreens_dn,"[0.0, 0.0014728324725323874, 0.002769917049663...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,0,513.78,517.061923,3.281923,15,2,514.170000,514.480698,0.310698,Bird5,0,LS110143.10.12.2015GeoGreens_dn,"[0.0, 4.4531726447127805e-05, 4.62385657049665...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,0,513.78,517.061923,3.281923,15,3,514.620783,515.310000,0.689217,Bird5,0,LS110143.10.12.2015GeoGreens_dn,"[-0.0, -3.2631556531601904e-06, -7.44037823939...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,0,513.78,517.061923,3.281923,15,4,515.355217,515.660180,0.304964,Bird5,0,LS110143.10.12.2015GeoGreens_dn,"[-0.0, -2.958346783314775e-05, -7.119856074752...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [47]:
max(note_df.note_len)

1.2611399019568807

In [64]:
import seaborn as sns
from tqdm.autonotebook import tqdm

In [68]:
def construct_rhythm_df(indv_df, syntactic_unit):
    
    dyadic_container = []
    
    ## for each bout
    bouts = [indv_df[indv_df['key'] == key] for key in np.unique(indv_df['key'].values)]
    for bout in tqdm(bouts):
        
        ## for each phrase
        phrases = [bout[bout['phrase_nb'] == phrase_nb] for phrase_nb in np.unique(indv_df['phrase_nb'].values)]
        
        for phrase in tqdm(phrases):
            if len(phrase) < 3: ## phrases with less than 3 notes cannot construct rhythm
                continue
            onsets = phrase['note_strt'].values
            syntactic_units = phrase[syntactic_unit].values

            ### DISCOVER ALL INTER-ONSET INTERVALS
            ## Use counter design
            i = 0
            intervals = []

            ## for every note onset
            for onset in onsets:

                ## if at last note, exit for loop
                if i == (len(onsets) - 1):
                    continue

                ## interval = next onset - current onset, add to interval list
                interval = onsets[i + 1] - onsets[i]
                intervals.append(interval)

                ## counter up
                i = i + 1


            ## Collect intervals into dyadic formation

            ## interval 1 does not include last interval
            intervals1 = list(intervals)
            del(intervals1[-1])

            ## interval 2 does not include first interval
            intervals2 = list(intervals)
            del(intervals2[0])

            ## Put intervals into datafrmae
            d = {'intervals1': intervals1, 'intervals2': intervals2}
            dyadic = pd.DataFrame(data = d)

            ## back propagate label and spec information

            ## note identity 1 // Delete last two values
            labels1 = list(syntactic_units)
            del(labels1[-1])
            del(labels1[-1])

            ## note identity 2 // Delete first and last value
            labels2 = list(syntactic_units)
            del(labels2[0])
            del(labels2[-1])

            ## note identity 3 // Delete first two values
            labels3 = list(syntactic_units)
            del(labels3[0])
            del(labels3[0])

            ## fill
            dyadic['label1'] = labels1
            dyadic['label2'] = labels2
            dyadic['label3'] = labels3

            ## Sort dyadic into short/long interval
            ### sort all intervals and calculate necessary components
            s_interval = []
            l_interval = []
            cycle_dur = []
            ratio_custom = []
            ratio_roeske = []

            ## for every dyadic
            for index, row in dyadic.iterrows():

                i1 = row['intervals1']
                i2 = row['intervals2']

                ## short long decider
                if i1 > i2:
                    s = i2
                    l = i1
                else:
                    s = i1
                    l = i2

                ## calculate components
                s_interval.append(s)
                l_interval.append(l)
                cycle_dur.append(s + l)
                ratio_roeske.append(i1/(i1+i2))
                ratio_custom.append(s/l)

            ## push into dyadic
            dyadic['s_interval'] = s_interval
            dyadic['l_interval'] = l_interval
            dyadic['cycle_dur'] = cycle_dur
            dyadic['ratio_roeske'] = ratio_roeske
            dyadic['ratio_custom'] = ratio_custom

            ## sort ascending by shortest cycle to longest cycle
            dyadic = dyadic.sort_values(by = ['cycle_dur'])

            ## put cycle rank into data frame
            dyadic['cycle_rank'] = dyadic['cycle_dur'].rank().astype(int)
            dyadic_container.append(dyadic)
    
    return pd.concat(dyadic_container)

In [69]:
construct_rhythm_df(note_df, 'key')

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

,intervals1,intervals2,label1,label2,label3,s_interval,l_interval,cycle_dur,ratio_roeske,ratio_custom,cycle_rank
7,0.047763,0.124941,LS110131.10.9.2015GeoPool_dn,LS110131.10.9.2015GeoPool_dn,LS110131.10.9.2015GeoPool_dn,0.047763,0.124941,0.172703,0.276559,0.382283,1
8,0.124941,0.053709,LS110131.10.9.2015GeoPool_dn,LS110131.10.9.2015GeoPool_dn,LS110131.10.9.2015GeoPool_dn,0.053709,0.124941,0.178649,0.699363,0.429873,2
9,0.053709,0.192387,LS110131.10.9.2015GeoPool_dn,LS110131.10.9.2015GeoPool_dn,LS110131.10.9.2015GeoPool_dn,0.053709,0.192387,0.246095,0.218243,0.279171,3
4,0.142577,0.114876,LS110131.10.9.2015GeoPool_dn,LS110131.10.9.2015GeoPool_dn,LS110131.10.9.2015GeoPool_dn,0.114876,0.142577,0.257453,0.553798,0.805713,4
6,0.255124,0.047763,LS110131.10.9.2015GeoPool_dn,LS110131.10.9.2015GeoPool_dn,LS110131.10.9.2015GeoPool_dn,0.047763,0.255124,0.302886,0.842308,0.187214,5
...,...,...,...,...,...,...,...,...,...,...,...
1,0.164585,0.358590,LS110640.10.1.2018Araluen_dn,LS110640.10.1.2018Araluen_dn,LS110640.10.1.2018Araluen_dn,0.164585,0.358590,0.523175,0.314589,0.458979,6
4,0.177804,0.350000,LS110640.10.1.2018Araluen_dn,LS110640.10.1.2018Araluen_dn,LS110640.10.1.2018Araluen_dn,0.177804,0.350000,0.527804,0.336876,0.508013,7
0,0.480513,0.355655,LS110640.10.1.2018Araluen_dn,LS110640.10.1.2018Araluen_dn,LS110640.10.1.2018Araluen_dn,0.355655,0.480513,0.836168,0.574661,0.740157,1
2,0.696176,0.273891,LS110640.10.1.2018Araluen_dn,LS110640.10.1.2018Araluen_dn,LS110640.10.1.2018Araluen_dn,0.273891,0.696176,0.970067,0.717657,0.393423,2


In [15]:
import numpy as np
import scipy.stats

## a standard calculation for confidence interval
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [20]:
bouts = []
for key in np.unique(note_df['key'].values):
    bouts.append(note_df[note_df['key'] == key].drop_duplicates('phrase_nb'))

In [22]:
bouts = pd.concat(bouts)

In [44]:
bouts

,phrase_nb,phrase_strt,phrase_end,phrase_len,note_cnt,note_nb,note_strt,note_end,note_len,indv,indvi,key,audio,rate,spectrogram
0,0,469.804892,472.345353,2.540460,14,0,469.804892,469.910903,0.106010,Bird5,0,LS110131.10.9.2015GeoPool_dn,"[0.0, -3.185799195362418e-05, -0.0001150179129...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
14,1,517.967926,519.610000,1.642074,11,0,517.967926,518.014062,0.046137,Bird5,0,LS110131.10.9.2015GeoPool_dn,"[-0.0, -2.3206200563167566e-05, -8.39456290806...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
25,2,556.528319,558.560000,2.031681,4,0,556.530000,556.657206,0.127206,Bird5,0,LS110131.10.9.2015GeoPool_dn,"[-0.0, -1.851759730656975e-06, -4.871229988041...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
29,3,577.729165,579.315047,1.585882,13,0,577.729165,577.793529,0.064363,Bird5,0,LS110131.10.9.2015GeoPool_dn,"[-0.0, -3.761239633067267e-07, 7.1627566541682...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
42,4,594.770000,596.290000,1.520000,5,0,594.770000,594.969535,0.199535,Bird5,0,LS110131.10.9.2015GeoPool_dn,"[0.0, 4.2028456376302964e-05, 9.10708905766474...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,213,7425.300000,7426.474002,1.174002,4,0,7425.300000,7425.448971,0.148971,Bird22,0,LS110640.10.1.2018Araluen_dn,"[-0.0, -7.633789248714511e-05, -6.415087404667...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1064,214,7431.580000,7432.711302,1.131302,2,0,7431.580000,7431.727575,0.147575,Bird22,0,LS110640.10.1.2018Araluen_dn,"[0.0, 0.0003904304498394064, 0.000701465462678...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1066,215,7441.120000,7442.649445,1.529445,9,0,7441.120000,7441.185206,0.065206,Bird22,0,LS110640.10.1.2018Araluen_dn,"[-0.0, 3.6092849206898186e-07, 2.5555794205416...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1075,216,7449.073765,7451.014733,1.940968,5,0,7449.073765,7449.300779,0.227014,Bird22,0,LS110640.10.1.2018Araluen_dn,"[0.0, 4.112177380773195e-07, 1.114693895332365...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [28]:
import butcherbird.rhythm as rhythm

/home/AD/j8xing/syntax_rhythm_pbb/butcherbird/rhythm.py:153: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [40]:
dyadic = rhythm.construct_rhythm_df(
    onsets = note_df['note_strt'].values, 
    syntactic_units = note_df['indv'], 
    expected_interval_range = [0, 1]
)

In [41]:
dyadic

,intervals1,intervals2,label1,label2,label3,s_interval,l_interval,cycle_dur,ratio_roeske,ratio_custom,cycle_rank
9722,0.018920,0.015573,Bird22,Bird22,Bird22,0.015573,0.018920,0.034493,0.548518,0.823094,1
30387,0.036773,0.003309,Bird5,Bird5,Bird5,0.003309,0.036773,0.040083,0.917441,0.089989,2
1010,0.034970,0.005312,Bird5,Bird5,Bird5,0.005312,0.034970,0.040282,0.868121,0.151913,3
72,0.039629,0.001440,Bird5,Bird5,Bird5,0.001440,0.039629,0.041068,0.964946,0.036327,4
18337,0.036661,0.018330,Bird26,Bird26,Bird26,0.018330,0.036661,0.054991,0.666667,0.500000,5
...,...,...,...,...,...,...,...,...,...,...,...
9485,0.555021,0.895000,Bird22,Bird22,Bird22,0.555021,0.895000,1.450021,0.382767,0.620135,24101
10585,0.707663,0.752358,Bird22,Bird22,Bird22,0.707663,0.752358,1.460021,0.484694,0.940593,24102
9403,0.670000,0.830021,Bird22,Bird22,Bird22,0.670000,0.830021,1.500021,0.446660,0.807209,24103
28160,0.555399,0.965593,Bird22,Bird22,Bird22,0.555399,0.965593,1.520993,0.365156,0.575190,24104


In [42]:
dyadic.intervals1.values

array([0.01892017, 0.03677345, 0.03496954, ..., 0.67      , 0.55539922,
       0.79241808])

In [43]:
mean_confidence_interval(dyadic.intervals1.values)

(0.19858242063066917, 0.19683800760570486, 0.20032683365563347)

In [7]:
indv = 'Bird26'

In [8]:
## Load up Note Dataframe

import pandas as pd
from pathlib2 import Path

from butcherbird.utils.paths import DATA_DIR

SAVE_LOC = 'interim/fine_labels/butcherbird_note_df_'+indv+'_fine.pickle'
SAVE_LOC = DATA_DIR / SAVE_LOC

note_df = pd.read_pickle(SAVE_LOC)

## Make a copy of recursively generated labels
#note_df['lasso_labels'] = note_df['recur_labels_id']

note_df

,phrase_nb,phrase_strt,phrase_end,phrase_len,note_cnt,note_nb,note_strt,note_end,note_len,indv,indvi,key,audio,rate,spectrogram,umap,hdbscan_labels,recur_labels,recur_labels_id,lasso_labels
0,0,2.530000,3.108940,0.578940,4,0,2.530000,2.565308,0.035308,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[0.0, 1.046653124410034e-05, 2.781446748131873...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[-7.5537066, 8.209083]",21,21|0|1.0|1.0|0.0|nan|nan|nan|nan,154,4
1,0,2.530000,3.108940,0.578940,4,1,2.582859,2.646042,0.063184,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[-0.0, -5.34641467372343e-06, -1.1688967129128...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[-0.6834063, 9.239816]",11,11|0|0.0|1.0|0.0|1.0|0.0|0.0|nan,14,2
2,0,2.530000,3.108940,0.578940,4,2,2.754858,2.877196,0.122338,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[0.0, 2.4410663172602654e-05, 5.92200898084064...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[-1.5902307, -12.39612]",0,0|0|0.0|0.0|1.0|0.0|nan|nan|nan,1,0
3,0,2.530000,3.108940,0.578940,4,3,2.942109,3.108940,0.166831,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[-0.0, -0.00015404379157812003, -0.00030952889...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[7.0395265, 3.0541244]",26,26|1|1.0|0.0|nan|nan|nan|nan|nan,206,19
4,1,7.660000,9.269842,1.609842,6,0,7.660000,7.802573,0.142573,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[0.0, 5.444124866051962e-05, 0.000182190384940...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[5.1442027, -3.2366111]",12,12|1|0.0|1.0|0.0|nan|nan|nan|nan,37,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10861,1426,7451.938961,7454.270021,2.331060,11,6,7453.230000,7453.347846,0.117846,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[0.0, 0.00020218813079315542, 0.00039304040322...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[5.835227, 3.594297]",25,25|1|1.0|1.0|0.0|0.0|nan|nan|nan,195,17
10862,1426,7451.938961,7454.270021,2.331060,11,7,7453.430000,7453.582862,0.152862,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[-0.0, -6.027892231941223e-05, -0.000147847971...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[5.1574388, 3.9381824]",25,25|0|0.0|0.0|nan|nan|nan|nan|nan,188,11
10863,1426,7451.938961,7454.270021,2.331060,11,8,7453.582862,7453.644792,0.061930,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[-0.0, -0.0007704722401279731, -0.002082758701...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[-0.96411866, 1.086623]",19,19|0|0.0|1.0|1.0|0.0|nan|nan|nan,118,9
10864,1426,7451.938961,7454.270021,2.331060,11,9,7453.786119,7453.917918,0.131800,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[-0.0, -2.3817842366836413e-06, -8.17539332897...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[2.2781262, -6.882173]",9,9|0|0.0|1.0|0.0|nan|nan|nan|nan,234,27


In [9]:
import numpy as np

In [10]:
df = []
for key in np.unique(note_df.key):
    df.append(note_df[note_df['key'] == key])

In [11]:
df[0]

,phrase_nb,phrase_strt,phrase_end,phrase_len,note_cnt,note_nb,note_strt,note_end,note_len,indv,indvi,key,audio,rate,spectrogram,umap,hdbscan_labels,recur_labels,recur_labels_id,lasso_labels
0,0,2.530000,3.108940,0.578940,4,0,2.530000,2.565308,0.035308,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[0.0, 1.046653124410034e-05, 2.781446748131873...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[-7.5537066, 8.209083]",21,21|0|1.0|1.0|0.0|nan|nan|nan|nan,154,4
1,0,2.530000,3.108940,0.578940,4,1,2.582859,2.646042,0.063184,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[-0.0, -5.34641467372343e-06, -1.1688967129128...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[-0.6834063, 9.239816]",11,11|0|0.0|1.0|0.0|1.0|0.0|0.0|nan,14,2
2,0,2.530000,3.108940,0.578940,4,2,2.754858,2.877196,0.122338,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[0.0, 2.4410663172602654e-05, 5.92200898084064...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[-1.5902307, -12.39612]",0,0|0|0.0|0.0|1.0|0.0|nan|nan|nan,1,0
3,0,2.530000,3.108940,0.578940,4,3,2.942109,3.108940,0.166831,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[-0.0, -0.00015404379157812003, -0.00030952889...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[7.0395265, 3.0541244]",26,26|1|1.0|0.0|nan|nan|nan|nan|nan,206,19
4,1,7.660000,9.269842,1.609842,6,0,7.660000,7.802573,0.142573,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[0.0, 5.444124866051962e-05, 0.000182190384940...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[5.1442027, -3.2366111]",12,12|1|0.0|1.0|0.0|nan|nan|nan|nan,37,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10861,1426,7451.938961,7454.270021,2.331060,11,6,7453.230000,7453.347846,0.117846,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[0.0, 0.00020218813079315542, 0.00039304040322...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[5.835227, 3.594297]",25,25|1|1.0|1.0|0.0|0.0|nan|nan|nan,195,17
10862,1426,7451.938961,7454.270021,2.331060,11,7,7453.430000,7453.582862,0.152862,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[-0.0, -6.027892231941223e-05, -0.000147847971...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[5.1574388, 3.9381824]",25,25|0|0.0|0.0|nan|nan|nan|nan|nan,188,11
10863,1426,7451.938961,7454.270021,2.331060,11,8,7453.582862,7453.644792,0.061930,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[-0.0, -0.0007704722401279731, -0.002082758701...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[-0.96411866, 1.086623]",19,19|0|0.0|1.0|1.0|0.0|nan|nan|nan,118,9
10864,1426,7451.938961,7454.270021,2.331060,11,9,7453.786119,7453.917918,0.131800,Bird26,0,LS110395.9.28.2017PalmRagonesi_dn,"[-0.0, -2.3817842366836413e-06, -8.17539332897...",48000,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[2.2781262, -6.882173]",9,9|0|0.0|1.0|0.0|nan|nan|nan|nan,234,27
